In [1]:
import requests
import json
import websocket
import traceback

class MyWebSocket:
    def __init__(self):
        self.received_data = None

    def on_error(self, ws, error):
        traceback_error = ''.join(traceback.format_tb(error.__traceback__))
        print(f"WebSocket error: {traceback_error}")

    def on_close(self, ws, a, b):
        print("WebSocket connection closed")

    def on_message(self, ws, message):
        message = json.loads(message)

        if 'errorCodes' in message.keys():
            print("Received disconnect signal, closing connection")
            print(message)
            ws.close()
        else:
            if message['finished'] == 'Y'  and message['message_id'] != '':
                self.received_data = message
                print("\nReceived disconnect signal, closing connection")
                ws.close()

            else:
                if message['response_content'] == '':
                    if message['wait_message'] == '추천 질문 리스트를 생성 중입니다':
                        print(f"\n\n- {message['wait_message']}")

                    elif message['wait_message'] == '답변을 생성 중입니다':
                        print(f"- {message['wait_message']}\n")

                    else:
                        print(f"- {message['wait_message']}")

                elif message['response_type'] == 'text':
                    print(message['response_content'], end='')
                else:
                    print(message['response_content'])

    def on_open(self, ws, data):
        print("WebSocket connection opened")
        ws.send(json.dumps(data))

    def get_received_data(self):
        return self.received_data

    def run(self, url, body):
        ws = websocket.WebSocketApp(url,
                                    on_message=self.on_message,
                                    on_error=self.on_error,
                                    on_close=self.on_close)

        ws.on_open = lambda ws: self.on_open(ws, body)
        ws.run_forever()

# DNS 설정

In [14]:
aia_dns = 'http://127.0.0.1:4000'
t2s_dns = 'http://127.0.0.1:5000'

# AIA

## login

In [16]:
id='hello'
password='bye'

In [17]:
response = requests.post(url = f'{aia_dns}/login', json = {"id":id, "password": password})

In [18]:
eval(response.text)

{'status': 'success', 'message': 'hello login success'}

## home

In [19]:
response = requests.post(url = f'{aia_dns}/home', json = {"id":"hello"})

In [20]:
eval(response.text)

{'id': 'hello',
 'favorites': 'N',
 'favorites_list': [],
 'svc_list': [{'svc_id': '1',
   'svc_nm': '사내 업무 정보 검색',
   'svc_dns': 'http://127.0.0.1:4000',
   'svc_ws': 'ws://127.0.0.1:4000',
   'backend_svc_id': 'aia',
   'user_type': '카드회사에서 고객의 문의를 상담해주고 있는 Call Center 상담원',
   'response_style': 'bullet point 형식으로 간결하게',
   'selected': 'Y'},
  {'svc_id': '2',
   'svc_nm': 'ERP 데이터 조회',
   'svc_dns': 'http://127.0.0.1:5000',
   'svc_ws': 'ws://127.0.0.1:5000',
   'backend_svc_id': 't2s',
   'user_type': '지점에서 지계수 데이터를 취합하는 현업 직원',
   'response_style': '질의에 적합한 SQL 쿼리와 데이터 추출',
   'selected': 'N'},
  {'svc_id': '3',
   'svc_nm': '이벤트 홍보 문구 및 이미지 생성',
   'svc_dns': 'http://127.0.0.1:6000',
   'svc_ws': 'ws://127.0.0.1:6000',
   'backend_svc_id': 'epa',
   'user_type': '현업 이벤트 기획 담당자',
   'response_style': '이벤트 컨텐츠 제작 요청 문서 초안 생성',
   'selected': 'N'}]}

## question

In [21]:
chat_id = '1234561ddddd'

In [22]:
ws_dns = aia_dns.replace('http:', 'ws:')

In [23]:
body = {"question": "Hello, WebSocket!"}

In [24]:
ws = MyWebSocket()
ws.run(f'{ws_dns}/question/{id}/{chat_id}', body)
message_id = ws.get_received_data()['message_id']

WebSocket connection opened
- 메뉴얼 탐색 중입니다
안녕하세요 수민님. 잘 부탁드립니다.테스트를 위해좀 더 내용을 늘려 보았습니다.[[FD팀 BL] 해외거래정지 3495/3496/2499 (그외
기타)](https://www.naver.com)
https://www.shinhancard.com/pconts/images/contents/card/plate/cdCreaditBJCBLP.png
FastAPI 로 Backend 구축하는 방법은?
소캣을 활용하여 스트리밍 구현하는 방법은?

Received disconnect signal, closing connection
WebSocket connection closed


## feedback

In [23]:
# id = response_json['id']
# chat_id = response_json['chat_id']
# message_id = response_json['message_id']
# message_id = 'kkk'

NameError: name 'response_json' is not defined

In [22]:
response = requests.post(url = f'{aia_dns}/feedback', json = {"id":id,
                                                   "chat_id" : chat_id,
                                                #    "message_id": 'kkk',
                                                   "message_id": message_id,
                                                   "feedback": "positive"
                                                   })

In [23]:
eval(response.text)

{'status': 'success', 'message': 'positive feedback log success'}

## T2S

## home

In [24]:
response = requests.post(url = f'{t2s_dns}/home', json = {"id":"hello"})

In [25]:
eval(response.text)

{'id': 'hello',
 'favorites': 'N',
 'favorites_list': [],
 'svc_list': [{'svc_id': '1',
   'svc_nm': '상담원 업무 정보 검색',
   'svc_dns': 'http://127.0.0.1:5000',
   'svc_ws': 'ws://127.0.0.1:5000',
   'backend_svc_id': 'aia',
   'user_type': '카드회사에서 고객의 문의를 상담해주고 있는 Call Center 상담원',
   'response_style': 'bullet point 형식으로 간결하게',
   'selected': 'Y'},
  {'svc_id': '2',
   'svc_nm': '지점 지계수 조회',
   'svc_dns': 'http://127.0.0.1:4000',
   'svc_ws': 'ws://127.0.0.1:4000',
   'backend_svc_id': 't2s',
   'user_type': '지점에서 지계수 데이터를 취합하는 현업 직원',
   'response_style': '질의에 적합한 SQL 쿼리와 데이터 추출',
   'selected': 'N'},
  {'svc_id': '3',
   'svc_nm': '이벤트 홍보 문구 및 이미지 생성',
   'svc_dns': 'http://127.0.0.1:6000',
   'svc_ws': 'ws://127.0.0.1:6000',
   'backend_svc_id': 'epa',
   'user_type': '현업 이벤트 기획 담당자',
   'response_style': '이벤트 컨텐츠 제작 요청 문서 초안 생성',
   'selected': 'N'}]}

## question

In [26]:
chat_id = '1234561'

In [27]:
ws_dns = t2s_dns.replace('http:', 'ws:')

In [28]:
body = {"question": "Hello, WebSocket!"}

In [29]:
ws = MyWebSocket()
ws.run(f'{ws_dns}/question/{id}/{chat_id}', body)
message_id = ws.get_received_data()

WebSocket connection opened
- 데이터 탐색 중입니다
쿼리  결과는  다음과 같습니다```
Select 
bye 
from 
dm.
hello
```- 쿼리 생성 중입니다
```
Select 
kitty 
from 
dm.
cat
```이상 연속쿼리 예제였습니다위버스 샵 결재 내역 뽑아줘
법인 카드 매출 뽑아줘

Received disconnect signal, closing connection
WebSocket connection closed


## feedback

In [30]:
response = requests.post(url = f'{t2s_dns}/feedback', json = {"id":id,
                                                   "chat_id" : chat_id,
                                                   "message_id": "kkk",
                                                #    "message_id": message_id,
                                                   "feedback": "positive"
                                                   })

In [31]:
eval(response.text)

{'errorCodes': ['CHAT_NOT_FOUND'],
 'detail': [{'type': 'not_found',
   'loc': ['body', 'chat_id'],
   'msg': '사용자의 채팅 기록을 찾을 수 없습니다.',
   'input': {}}]}